In [1]:
import pandas as pd
dataset = pd.read_csv("cleaned_dataset.csv", index_col = 0)
dataset.head()

text  verdict
0   case of gestur jónsson and ragnar halldór hal...        0
1   case of selahatti̇n demi̇rtaş v. turkey (no. ...        1
2   case of usmanov v. russia\n43936/18   |   jud...        1
3   case of plokhovy v. russia\n45024/07   |   ju...        1
4   case of j.a. v. switzerland\n6325/15   |   ju...        0

In [2]:
import spacy
import unidecode
from pycontractions import Contractions
import contractions
import gensim.downloader as api
from tqdm.notebook import tqdm
# load spacy model, can be "en_core_web_sm" as well
nlp = spacy.load('en_core_web_sm')
# Choose model accordingly for contractions function
model = api.load("glove-twitter-25")
# model = api.load("glove-twitter-100")
# model = api.load("word2vec-google-news-300")
cont = Contractions(kv_model=model)
cont.load_models()

In [3]:
#remove newline characters
#https://stackoverflow.com/questions/44227748/removing-newlines-from-messy-strings-in-pandas-dataframe-cells
dataset["text"] = dataset["text"].replace('\n',' ', regex=True)

In [4]:
#https://stackoverflow.com/questions/39782418/remove-punctuations-in-pandas
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

dataset.text = dataset.text.apply(remove_punctuations)

In [5]:
#remove stopwords
#https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
set(stopwords.words('english'))

#https://stackabuse.com/removing-stop-words-from-strings-in-python/
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
all_stopwords.remove('nor')
all_stopwords.remove('no')
all_stopwords.append('url')

for i in tqdm(range(len(dataset.text))):
    text_tokens = word_tokenize(dataset.text[i])
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
    filtered_text = (" ").join(tokens_without_sw)
    dataset.text[i] = filtered_text

C:\Users\tabbe\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
#lemmatize the texts
#https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79
from nltk.tokenize.treebank import TreebankWordDetokenizer

dummylist = []
for i in tqdm(range(len(dataset.text))):
    string = str(dataset.text[i])
    doc = nlp(string)
    mytokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in doc]
    string2 = TreebankWordDetokenizer().detokenize(mytokens)
    dummylist.append(string2)
    
dataset.text = dummylist

In [7]:
#expand contracted words (e.g. don't becomes do not)
#use one option

""""#slow (haven't completed yet)
def expand_contractions(text):
    text = list(cont.expand_texts([text], precise=True))[0]
    return text

dataset.text = dataset.text.apply(expand_contractions)"""

#or
#https://www.geeksforgeeks.org/nlp-expand-contractions-in-text-processing/

for i in range(len(dataset.text)):
    dataset.text[i] = contractions.fix(dataset.text[i])


C:\Users\tabbe\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [8]:
#replace accented chars with standard letters
def remove_accented_chars(text):
    text = unidecode.unidecode(text)
    return text

dataset.text = dataset.text.apply(remove_accented_chars)

In [9]:
#remove extra whitespaces
def remove_whitespace(text):
    text = text.strip()
    return " ".join(text.split())

dataset.text = dataset.text.apply(remove_whitespace)

In [12]:
#write the cleaned dataset to new .csv file
dataset.to_csv("processed_dataset.csv", header=["text","verdict"])

In [2]:
import pandas as pd
df = pd.read_csv("processed_dataset.csv", index_col=0)

In [6]:
dummylist = df['verdict'].tolist()
print(dummylist.count(0))
print(dummylist.count(1))

3548
9534
